### Random Forest - Regression

In [18]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('medical_insurance.csv')
df

,age,gender,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   gender    1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [8]:
df['gender'].replace({'male':1, 'female':0}, inplace = True)

In [9]:
df['smoker'].replace({'yes':1, 'no':0}, inplace = True)

In [ ]:
df = pd.get_dummies(df, columns=['region'], dtype = int)

In [13]:
df

,age,gender,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,10600.54830,0,1,0,0
1334,18,0,31.920,0,0,2205.98080,1,0,0,0
1335,18,0,36.850,0,0,1629.83350,0,0,1,0
1336,21,0,25.800,0,0,2007.94500,0,0,0,1


### Model Training

In [15]:
x = df.drop('charges', axis = 1)
y = df['charges']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=12)

In [17]:
rf_reg = RandomForestRegressor(n_jobs=-1)
rf_reg.fit(x_train, y_train)

RandomForestRegressor(n_jobs=-1)

#### Evaluation

In [24]:
# Training Data evaluvation
y_pred_train = rf_reg.predict(x_train)
mse = mean_squared_error(y_train ,y_pred_train)
print("MSE :",mse)

rmse = np.sqrt(mse)
print("RMSE :",rmse)

r2_value = r2_score(y_train ,y_pred_train)
print("R-squared Value :",r2_value)

MSE : 3324754.2112603206
RMSE : 1823.3908553188262
R-squared Value : 0.9774083405769849


In [26]:
# Testing Data evaluvation
y_pred_test = rf_reg.predict(x_test)
mse = mean_squared_error(y_test ,y_pred_test)
print("MSE :",mse)

rmse = np.sqrt(mse)
print("RMSE :",rmse)

r2_value = r2_score(y_test ,y_pred_test)
print("R-squared Value :",r2_value)

MSE : 25077702.832461372
RMSE : 5007.764254880752
R-squared Value : 0.8258630969297842


### Hypeparameter Tuning

In [30]:
rf_reg = RandomForestRegressor(n_jobs=-1, random_state =1)

hyp_grid = {"n_estimators" : range(20,80,15),
            "criterion" : ['squared_error', 'absolute_error'],
            "max_depth" : range(5,9),
            "min_samples_split" : range(10,15),
            "min_samples_leaf" : range(7,12)}

gscv_rf_reg = GridSearchCV(rf_reg, hyp_grid, cv=3, n_jobs=-1, verbose=1)
gscv_rf_reg.fit(x_train, y_train)

Fitting 3 folds for each of 800 candidates, totalling 2400 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(n_jobs=-1), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error'],
                         'max_depth': range(5, 9),
                         'min_samples_leaf': range(7, 12),
                         'min_samples_split': range(10, 15),
                         'n_estimators': range(20, 80, 15)},
             verbose=1)

In [32]:
rf_reg = gscv_rf_reg.best_estimator_
rf_reg.fit(x_train, y_train)

RandomForestRegressor(max_depth=5, min_samples_leaf=9, min_samples_split=14,
                      n_estimators=20, n_jobs=-1)

In [34]:
# Training Data evaluvation
y_pred_train = rf_reg.predict(x_train)
mse = mean_squared_error(y_train ,y_pred_train)
print("MSE :",mse)

rmse = np.sqrt(mse)
print("RMSE :",rmse)

r2_value = r2_score(y_train ,y_pred_train)
print("R-squared Value :",r2_value)

MSE : 16551913.96955611
RMSE : 4068.4043517767636
R-squared Value : 0.8875299708072228


In [35]:
# Testing Data evaluvation
y_pred_test = rf_reg.predict(x_test)
mse = mean_squared_error(y_test ,y_pred_test)
print("MSE :",mse)

rmse = np.sqrt(mse)
print("RMSE :",rmse)

r2_value = r2_score(y_test ,y_pred_test)
print("R-squared Value :",r2_value)

MSE : 23968145.238812376
RMSE : 4895.727243098044
R-squared Value : 0.8335677469301015
